In [9]:
!pip install lightning

In [10]:
import os
import torch
import torch.nn.functional as F
import lightning as L

from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [11]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28*28, 64), 
                                nn.ReLU(),
                                nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28*28))

    def forward(self, x):
        return self.l1(x)

In [12]:
# Repeated code - why?
class BasicAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        x, _ = batch
        
        # Flatten
        x = x.view(x.size(0), -1)

        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)

        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = F.mse_loss(x_hat, x)
        self.log("val_loss", val_loss)
        

    def test_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = F.mse_loss(x_hat, x)
        self.log("test_loss", test_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [15]:
transform = transforms.ToTensor()
train_set = dataset.MNIST(root="MNIST", download=True, train=True, transform=transform)
test_set = dataset.MNIST(root="MNIST", download=True, train=False, transform=transform)

In [ ]:
# train-val split
train_set_size = int(len(train_set) * 0.8)
valid_set_size = len(train_set) - train_set_size

seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(train_set, [train_set_size, valid_set_size], generator = seed)

data_sets = [train_set, valid_set, test_set]

In [ ]:
# Init data loaders
init_loader = lambda x: DataLoader(x, num_workers=4, pin_memory=True)
train_loader, valid_loader, test_loader = [init_loader(x) for x in data_sets]

In [ ]:
autoencoder = BasicAutoEncoder(Encoder(), Decoder())

trainer = L.Trainer(accelerator="gpu")
trainer.fit(autoencoder, train_loader, valid_loader)

INFO: Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | Encoder | 50.4 K | train
1 | decoder | Decoder | 51.2 K | train
--------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB

Training: |          | 0/? [00:00<?, ?it/s]

In [14]:
os.cpu_count()

4

In [ ]:
trainer.test(model, dataloaders=)